# QE input script generator
This will automate the process of writing input scripts for Quantum Espresso to do the structural optimization of functionalized mofs using the pw.x (plane wave) package and pseudopotential files. 

**NOTE:** many default arguements will be preset.

In [8]:
using PorousMaterials
using Printf

In [17]:
PATH_TO_PSEUDOPOTENTIALS = joinpath(pwd(),"QE_scripts", "pseudo",
    "SSSP_PBEsol_efficiency_pseudos")

PATH_TO_MOFS = joinpath(pwd(), "NiPyC2")

"/home/ng/Documents/Graduate_Research/Cory_Simon/DTRA/pseudo/SSSP_PBEsol_efficiency_pseudos"

In [12]:
# populate dictionary with names of pseudopotential files
# note: may need to ignore case sensitivity?
# ex: atomic_psp[:Ni] = ["ni_pbesol_v1.4.uspp.F.UPF"]

psp_lib = Dict{Symbol, String}()

Dict{Symbol,String} with 0 entries

In [ ]:
# create a dictionary consisting of input arguements
# needed for the script and set default values
# then populate with values specific to each material

In [ ]:
# import the functionalized mofs 

In [ ]:
filename = "pw." * crystal.name * "vc-relax.in"

In [ ]:
f = open(filename, "w")
# @printf(f, "%s %f\n", xtal.atoms.species[s], xtal.atoms.x[s, 1])

In [ ]:
@printf(f,
    """
    &CONTROL
    calculation = 'vc-relax'
    prefix = '%s'
    pseudo_dir = 'pseudo/SSSP_PBEsol_efficiency_pseudos/'
    wf_collect=.false.
    /
    &SYSTEM
    ibrav = 0,
    nat = %.0f,
    ntyp = %.0f,
    tot_charge = %f,
    ecutwfc = %f,
    vdw_corr='grimme-d2'
    /
    &ELECTRONS
    diagonalization='david',
    /
    &IONS
    ion_dynamics='bfgs',
    /
    &CELL
    cell_dynamics='bfgs'
    cell_dofree='xyz'
    /
    ATOMIC_SPECIES
    #symbol #atomic_mass #pseudo
    CELL_PARAMETERS (angstrom)
    # matrix
    ATOMIC_POSITIONS
    #symbol #x #y #z
    K_POINTS {gamma}
    """,
    args)

In [1]:
close(f)

UndefVarError: UndefVarError: f not defined

In [ ]:
# how do I write the cell parameters and
# atomic positions as a formatted string?